In [1]:
import redis
import os
import threading
import time
from datetime import datetime
import json
from elasticsearch import Elasticsearch
from dotenv import dotenv_values
import requests
import traceback
import logging
import sys

sys.path.append("/home/haoran/projects/roobi-scraper/platforms")

from kibsons.call_xbytes import get_products as kibsons_get_products

config = dotenv_values(".env")

log_level = config.get('LOG_LEVEL', 'INFO').upper()
logging.basicConfig(level=log_level, format='%(asctime)s - %(levelname)s - %(message)s')


session = requests.Session()

r = redis.Redis(host=config["REDIS_HOST"], port=6379, decode_responses=True)


In [ ]:
es_host = config["ES_HOST"]
#DB_UPDATE_API = config["DB_UPDATE_API"]
ELASTIC_PASSWORD=config["ELASTIC_PASSWORD"]
es_client = Elasticsearch(es_host,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

store_status_processor = StoreStatus()

In [865]:
# r.keys("*")
name = "realtime_stock_{platform}_products".format(platform="kbs")
res = r.blpop(name, timeout=1)
print(res)

('realtime_stock_kbs_products', '{"_index": "products_geo_info_8a43a1220467fff_20241114", "_id": "8f5d08893e194cd1b17885111159f36a", "_score": null, "_source": {"product_sale_price": 15.2, "product_id_orig": 152046, "product_name": "Prawn Tempura Ebi Fry 10 Jumbo Prawns", "platform": "kbs", "store_id": "kbs", "store_name": "kibsons", "category": "Frozen", "vendor_id": null, "bid": null, "shop_type": null, "product_id": "4f2f8e64bd76118218df90188ae81b27", "popularity": 1}, "sort": [15.2], "latitude": "25.087054289271904", "longitude": "55.14804929494858"}')


In [874]:
message = res
hit = json.loads(message[1])
source = hit["_source"]
platform_code = source["platform"]

# should put it in exception final
# 获取任务的索引和产品 ID
index = hit["_index"]
product_id_for_matching = source["product_id"]
# 在 Redis 中减少任务计数
r.decr(f'rtp_task:{index}:{product_id_for_matching}:count')

if source.get("shop_type") != None:
    shop_type = int(source["shop_type"])
else:
    shop_type = source.get("shop_type")
platform_name_conv = {"cf_std":"carrefour","cf_now":"carrefour","nn":"nownow","tb":"talabat","is":"instashop","kbs":"kibsons"}

one_data = {"platform":platform_name_conv[platform_code],
            "store_id":source["store_id"],
            "product_id":source["product_id_orig"],
            "product_name":source["product_name"],
            "superCategoryCode":source.get("superCategoryCode"),
            # "store_name":source["store_name"],
            "category":source.get("category"), # for kibsons
            "vendor_id":source.get("vendor_id"),
            "shop_type":shop_type,
            "bid":source.get("bid"),
            "lat":hit["latitude"],
            "lng":hit["longitude"],
            "delivery_time":"Now" # Now or STANDARD
            }

In [ ]:
one_data
# source
# hit
# message

('realtime_stock_kbs_products',
 '{"_index": "products_geo_info_8a43a1220467fff_20241114", "_id": "8f5d08893e194cd1b17885111159f36a", "_score": null, "_source": {"product_sale_price": 15.2, "product_id_orig": 152046, "product_name": "Prawn Tempura Ebi Fry 10 Jumbo Prawns", "platform": "kbs", "store_id": "kbs", "store_name": "kibsons", "category": "Frozen", "vendor_id": null, "bid": null, "shop_type": null, "product_id": "4f2f8e64bd76118218df90188ae81b27", "popularity": 1}, "sort": [15.2], "latitude": "25.087054289271904", "longitude": "55.14804929494858"}')

In [886]:
category = one_data.get("category")
# print(category)
# xbytes_res = kibsons_get_products(category)
# producsts = xbytes_res.get("Products",{}).get(category,[])
producsts[-20:]

[{'Title': 'Ice Cream',
  'Actual Price': 25,
  'Discount Price': 0,
  'Price Per Size': 'pack',
  'Description': 'Choc & Vanilla - 360ml',
  'Brand': 'Bulla',
  'Featured': [],
  'Popularity Score': 0,
  'Origin Country': 'Australia',
  'IsFavorite': False,
  'Product ID': 210478,
  'Product Code': 'ICCSTAUBL360SF',
  'InStock': 'OUT OF STOCK',
  'ImageURL': 'https://kibsecomimgstore.blob.core.windows.net/products/display/HPL_ICCSTAUBL360SF_20240517112655.jpg'},
 {'Title': 'Salmon Fishcakes',
  'Actual Price': 28,
  'Discount Price': 0,
  'Price Per Size': 'pack',
  'Description': 'Atlantic Salmon - 270g',
  'Brand': 'Donegal Catch',
  'Featured': [],
  'Popularity Score': 0,
  'Origin Country': 'Ireland',
  'IsFavorite': False,
  'Product ID': 202670,
  'Product Code': 'SALSAIEGI270SF',
  'InStock': 'OUT OF STOCK',
  'ImageURL': 'https://kibsecomimgstore.blob.core.windows.net/products/display/HPL_SALSAIEGI270SF_20231211135302.jpg'},
 {'Title': 'Potato Fries',
  'Actual Price': 26.9,


In [884]:
one_data

{'platform': 'kibsons',
 'store_id': 'kbs',
 'product_id': 152046,
 'product_name': 'Prawn Tempura Ebi Fry 10 Jumbo Prawns',
 'superCategoryCode': None,
 'category': 'Frozen',
 'vendor_id': None,
 'shop_type': None,
 'bid': None,
 'lat': '25.087054289271904',
 'lng': '55.14804929494858',
 'delivery_time': 'Now'}

In [ ]:
# test nownow
import sys
sys.path.append("/home/haoran/projects/roobi-scraper/platforms")
from nownow.call_x_bytes import nownow_list_stores, nownow_store_details, nownow_produt_details


In [3]:
product_name = "Almarai Full Fat Fresh Milk"
store_id = "LMY4LRB96P"
xbytes_res = nownow_produt_details(product_name,store_id)
xbytes_res

{'request_log': {'requests_url': 'https://now-now.xbyteapi.com/now_now_uae_api?api_key=ggs5k8a6p9x7t&store_id=LMY4LRB96P&product_name=Almarai+Full+Fat+Fresh+Milk&endpoint=nownow_uae_app_master_data&endpoint_category=nownow_uae_app_product_searching', 'request_time': '2025-02-02 19:34:13.367779+00:00', 'process_time': '4.130942106246948'}, 'results': {'status': 200, 'data': {'products': [{'Search_Product_Name': 'Almarai Full Fat Fresh Milk', 'Store_Code': 'LMY4LRB96P', 'Total_results': 1728, 'Product_Code': 'P8986245940336505A', 'Name': 'Al Marai Fresh Milk With Vitamins, Full Fat', 'Product_Desc': '', 'Images': 'https://f.nooncdn.com/nownow/shop/partner/partner_9999/d31971c24c32682335e6d3ef690bc7d7_0.jpg', 'Category': 'milk', 'SubCategory': 'freshmilk', 'Price': 1.99, 'Price_Discount': 0.0, 'Discounted_Price': 1.99, 'Maximum_quantity': 15, 'Size': '180', 'Size_Unit': 'ml'}, {'Search_Product_Name': 'Almarai Full Fat Fresh Milk', 'Store_Code': 'LMY4LRB96P', 'Total_results': 1728, 'Produc

{'request_log': {'requests_url': 'https://now-now.xbyteapi.com/now_now_uae_api?api_key=ggs5k8a6p9x7t&store_id=LMY4LRB96P&product_name=Almarai+Full+Fat+Fresh+Milk&endpoint=nownow_uae_app_master_data&endpoint_category=nownow_uae_app_product_searching',
  'request_time': '2025-02-02 19:34:13.367779+00:00',
  'process_time': '4.130942106246948'},
 'results': {'status': 200,
  'data': {'products': [{'Search_Product_Name': 'Almarai Full Fat Fresh Milk',
     'Store_Code': 'LMY4LRB96P',
     'Total_results': 1728,
     'Product_Code': 'P8986245940336505A',
     'Name': 'Al Marai Fresh Milk With Vitamins, Full Fat',
     'Product_Desc': '',
     'Images': 'https://f.nooncdn.com/nownow/shop/partner/partner_9999/d31971c24c32682335e6d3ef690bc7d7_0.jpg',
     'Category': 'milk',
     'SubCategory': 'freshmilk',
     'Price': 1.99,
     'Price_Discount': 0.0,
     'Discounted_Price': 1.99,
     'Maximum_quantity': 15,
     'Size': '180',
     'Size_Unit': 'ml'},
    {'Search_Product_Name': 'Almarai